In [ ]:
# Import all the dependencies

import nvidia
import os
import time
import torch
import transformers

from datasets import load_dataset
from random import randint
from vllm import LLM, SamplingParams

In [ ]:
cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir

In [ ]:
# Load dataset from the hub
test_dataset = load_dataset("samsum", split="test")

In [ ]:
# Load the model to vLLM. Please change the path below according to your project
llm = LLM(model="/mnt/artifacts/falcon_7b_model_adapter",trust_remote_code=True)

In [ ]:
# select a random test sample
# sample = test_dataset[randint(0, len(test_dataset))]

# Change the index to select a different sample
sample = test_dataset[5]

# format sample
prompt_template = f"Summarize the chat dialogue:\n{{dialogue}}\n---\nSummary:\n"

test_sample = prompt_template.format(dialogue=sample["dialogue"])

print(test_sample)

In [ ]:
max_tokens = 50

sampling_params = SamplingParams(
        temperature=1.0,
        top_p=1,
        max_tokens=max_tokens,
    )
start_time = time.perf_counter()
outputs = llm.generate(test_sample, sampling_params)
end_time = time.perf_counter()
generated_text = outputs[0].outputs[0].text
print(f"{generated_text!r}")

In [ ]:
print(f'\n vLLM took {round(end_time - start_time, 3)} sec and generated {round(max_tokens / (end_time - start_time),3)}  tokens/sec')

In [ ]:
%%timeit -n 1 -r 30
outputs = llm.generate(test_sample, sampling_params)
generated_text = outputs[0].outputs[0].text